In [0]:
import torch as tr
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
import pandas
class BOW():
    def __init__(self,dataset,strip_accents,stoplist,weighting,ngram = None):
        #autocorregir -> lematizar -> borrar signos, carac especiales, stopwords, pasar a minuscula
        #x_text_auto = preprocesamiento.Autocorrector(dataset.values)
        #x_text_lem = preprocesamiento.Lematizar(dataset)
        #x_text_lem = x_text_lem[:,1]
        #x_text_lem = dataset[:,1] esto anda
        if weighting:
            if ngram: 
                vectorizer = TfidfVectorizer(strip_accents=strip_accents,stop_words=stoplist,ngram_range=ngram) 
            else:
                vectorizer = TfidfVectorizer(strip_accents=strip_accents,stop_words=stoplist)   
        else:
            if ngram:
                vectorizer = CountVectorizer(strip_accents=strip_accents,stop_words=stoplist,ngram_range=ngram)     
            else: 
                vectorizer = CountVectorizer(strip_accents=strip_accents,stop_words=stoplist)
        #X = vectorizer.fit_transform(x_text_lem)  
        X = vectorizer.fit_transform(dataset)
        self.vectorizer = vectorizer
        self.X = X

    def get_vocab(self):
        return(self.vectorizer.get_feature_names())

In [0]:
import numpy as np
import torch
import matplotlib.pyplot as plt
#from torchvision import datasets
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from nltk.corpus import stopwords
#from prueba2 import MLP
!pip install skorch
import skorch
from skorch import NeuralNetClassifier,NeuralNet
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score
from mpl_toolkits import mplot3d
import scipy.interpolate as interp
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive
drive.mount("/content/drive")

     |████████████████████████████████| 112kB 48.4MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#cargar csv con preguntas preprocesadas
print("-----------CARGANDO/CREANDO PREGUNTAS PREPROCESADAS--------------")
#correctedData = preprocesamiento.preprocesar(dataset.values,1) #Dataset lematizado, descomentar en caso de crear otro dataset 
correctedData = pd.read_csv("preprocessedQuestions_stem_completadas.csv",delimiter=',',header=None) #comentar esta linea en caso de descomentar la anterior
print("----------FINISHED CARGANDO/CREANDO PREGUNTAS PREPROCESADAS-------------\n")

#obtener datos utiles sobre el dataset
labels = correctedData.values[:,0]
print("Shape de corrected data: ",correctedData.shape)
cantidad_labels = correctedData.values[len(correctedData.values)-1,0] + 1
cantidad_preg = correctedData.shape[0]
print("Cantidad de clases: ",cantidad_labels)
print("Cantidad de patrones: ",cantidad_preg)
print("Lista de clases para cada pregunta :",labels)
correctedData = correctedData.values


-----------CARGANDO/CREANDO PREGUNTAS PREPROCESADAS--------------
----------FINISHED CARGANDO/CREANDO PREGUNTAS PREPROCESADAS-------------

Shape de corrected data:  (1527, 2)
Cantidad de clases:  106
Cantidad de patrones:  1527
Lista de clases para cada pregunta : [0 0 0 ... 105 105 105]


In [0]:
import nltk
nltk.download('stopwords')
stoplist = stopwords.words('spanish')
text = correctedData[:,1]
bow_unigram = BOW(text,'ascii',stoplist,weighting=False, ngram=(1,3))

print("-------------CREANDO Ytest e Ytrain (groundtruth de cada subconjunto), Xtext_test y Xtext_train-------------------")
Y = np.zeros((cantidad_preg),dtype=np.int64)
for i in range(cantidad_preg):
    Y[i] = correctedData[i,0]

Y = torch.from_numpy(Y)
print(Y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
-------------CREANDO Ytest e Ytrain (groundtruth de cada subconjunto), Xtext_test y Xtext_train-------------------
tensor([  0,   0,   0,  ..., 105, 105, 105])


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['eramos', 'estabamos', 'estais', 'estan', 'estara', 'estaran', 'estaras', 'estare', 'estareis', 'estaria', 'estariais', 'estariamos', 'estarian', 'estarias', 'esteis', 'esten', 'estes', 'estuvieramos', 'estuviesemos', 'fueramos', 'fuesemos', 'habeis', 'habia', 'habiais', 'habiamos', 'habian', 'habias', 'habra', 'habran', 'habras', 'habre', 'habreis', 'habria', 'habriais', 'habriamos', 'habrian', 'habrias', 'hayais', 'hubieramos', 'hubiesemos', 'mas', 'mia', 'mias', 'mio', 'mios', 'seais', 'sera', 'seran', 'seras', 'sere', 'sereis', 'seria', 'seriais', 'seriamos', 'serian', 'serias', 'si', 'tambien', 'tendra', 'tendran', 'tendras', 'tendre', 'tendreis', 'tendria', 'tendriais', 'tendriamos', 'tendrian', 'tendrias', 'teneis', 'tengais', 'tenia', 'teniais', 'teniamos', 'tenian', 'tenias', 'tuvier

In [0]:
D_in = bow_unigram.X.shape[1]
D_out = cantidad_labels
print(D_in)
print(D_out)

5254
106


In [0]:
print("-----------------INIT TRAINING PROCESS---------------------")
parameters = {    
'hidden_layer_sizes' : [(4000,1000,500),(4000),(4000,1500,300),(5000,1000)],
#'n_layers' : [1,2,3,4],
#'dropout' : [0.2,0.3,0.4,0.5,0.6],
#'bidirectional' : [True,False],
'max_iter' : [50,100,200,300],
'alpha' : [0.00001,0.0001,0.001,0.1,0.2],
'batch_size' : [1500,750,500,300]
}

net = MLPClassifier(hidden_layer_sizes=(100))
candidatos = 10
gs = RandomizedSearchCV(net,parameters,verbose=2,n_jobs=-1,cv=10,scoring='balanced_accuracy',n_iter=candidatos)
print(type(bow_unigram.X))
X_numpy = bow_unigram.X.toarray()
print(type(X_numpy))
print(type(Y))
Y_numpy = Y.numpy()
X_train,X_test,y_train,y_test = train_test_split(X_numpy,Y_numpy,shuffle=True,stratify=Y,test_size=0.1,random_state=12)

Y_numpy = np.asarray(y_train)

gs.fit(X_train,Y_numpy)

-----------------INIT TRAINING PROCESS---------------------
<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
hidden_layer_sizes = []
max_epochs = []
alpha= []
batch_size=[]
std=[]
score=[]
y_test_tensor = torch.LongTensor(y_test)
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            score.append(results['mean_test_score'][candidate])
            std.append(results['std_test_score'][candidate])
            hidden_layer_sizes.append(results['params'][candidate]['hidden_layer_sizes'])
            batch_size.append(results['params'][candidate]['batch_size'])
            alpha.append(results['params'][candidate]['alpha'])
            max_epochs.append(results['params'][candidate]['max_iter'])


report(gs.cv_results_,candidatos)  

print("Batch size: ",batch_size)
print("Hidden Dim: ",hidden_layer_sizes)
print("Max epochs: ",max_epochs)
print("Alpha: ",alpha)
print("Score: ",score)
print("std: ",std)


Model with rank: 1
Mean validation score: 0.776 (std: 0.037)
Parameters: {'max_iter': 50, 'hidden_layer_sizes': 500, 'batch_size': 300, 'alpha': 0.001}

Model with rank: 2
Mean validation score: 0.773 (std: 0.037)
Parameters: {'max_iter': 100, 'hidden_layer_sizes': 550, 'batch_size': 300, 'alpha': 1e-05}

Model with rank: 3
Mean validation score: 0.773 (std: 0.032)
Parameters: {'max_iter': 50, 'hidden_layer_sizes': 600, 'batch_size': 300, 'alpha': 0.001}

Model with rank: 4
Mean validation score: 0.771 (std: 0.030)
Parameters: {'max_iter': 75, 'hidden_layer_sizes': 550, 'batch_size': 300, 'alpha': 0.0001}

Model with rank: 5
Mean validation score: 0.769 (std: 0.027)
Parameters: {'max_iter': 100, 'hidden_layer_sizes': 500, 'batch_size': 300, 'alpha': 1e-05}

Model with rank: 6
Mean validation score: 0.768 (std: 0.030)
Parameters: {'max_iter': 50, 'hidden_layer_sizes': 500, 'batch_size': 300, 'alpha': 1e-05}

Batch size:  [300, 300, 300, 300, 300, 300]
Hidden Dim:  [500, 550, 600, 550, 5

In [0]:
hidden_dim = hidden_layer_sizes
num_layers = list(map(lambda x: 1 if type(x)==int else len(x), hidden_dim))
hidden_dim = list(map(lambda x: x if type(x)==int else np.average(x), hidden_dim))
#num_layers = [1,1,1,2,2,2,1,1,2,1,1,3,2,3,1]
ejex = hidden_dim #Da error esto, uso promedio de neuronas o qué onda? 
ejey = num_layers
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,40)
ax.set_xlabel('Neuronas')
ax.set_ylabel('Capas')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de la arquitectura')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba3_Neuronas_Capas_Precision.svg")
#/////////////////////////////
#////////////////////////////

ejex = hidden_dim
ejey = num_layers
ejez = std

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,40)
ax.set_xlabel('Neuronas')
ax.set_ylabel('Capas')
ax.set_zlabel('Std')
ax.set_title('Std en función de la arquitectura')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba3_Neuronas_Capas_STD.svg")
#/////////////////////////////
#////////////////////////////

ejex = max_epochs
ejey = batch_size
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Epocas')
ax.set_ylabel('Batch')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de tamaño de batch y épocas')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba3_Epocas_Batch_Precision.svg")

#/////////////////////////////
#////////////////////////////

ejex = max_epochs
ejey = batch_size
ejez = std

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Epocas')
ax.set_ylabel('Batch')
ax.set_zlabel('Std')
ax.set_title('Std en función de tamaño de batch y épocas')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba3_Epocas_Batch_STD.svg")

#/////////////////////////////
#////////////////////////////

ejex = alpha
ejey = max_epochs
ejez = score

plotx,ploty, = np.meshgrid(np.linspace(np.min(ejex),np.max(ejex),10),\
                           np.linspace(np.min(ejey),np.max(ejey),10))
plotz = interp.griddata((ejex,ejey),ejez,(plotx,ploty),method='cubic')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(35,120)
ax.set_xlabel('Alpha')
ax.set_ylabel('Epocas')
ax.set_zlabel('Precisión')
ax.set_title('Precisión en función de la regularización L2')

surf = ax.plot_surface(plotx,ploty,plotz,vmin=np.nanmin(ejez),vmax = np.nanmax(ejez),cstride=1,rstride=1,cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig("prueba3_Alpha_Epocas_Precision.svg")

probs = gs.best_estimator_.predict_proba(X_test)
#print(probs.shape)

# get training and validation loss
#epochs = [i for i in range(len(gs.best_estimator_.history))]
epochs = [i for i in range(max_epochs[0])]
#train_loss = gs.best_estimator_.history[:,'train_loss']
train_loss = gs.best_estimator_.loss_curve_

#valid_loss = gs.best_estimator_.history[:,'valid_loss']
acc = balanced_accuracy_score(y_test_tensor,np.argmax(probs,axis=1))
print("tasa de acierto obtenida: ",acc)
fig1 = plt.figure()
plt.plot(epochs,train_loss,'g-')
#plt.plot(epochs,valid_loss,'r-')
plt.title('Curvas del error en el entrenamiento')
plt.xlabel('Epocas')
plt.ylabel('Error (Cross Entropy Loss)')
plt.legend(['Entrenamiento'])
plt.savefig("prueba3_TrainVsVal.svg")


QhullError: ignored

In [0]:
acc = balanced_accuracy_score(y_test_tensor,np.argmax(probs,axis=1))
print("tasa de acierto obtenida: ",acc)

tasa de acierto obtenida:  0.7987421383647799
